[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ACS-IC-labs/IC-labs/blob/main/labs/lab04/lab4.ipynb)

# Laboratorul 04 - PRFs, PRPs, SPNs


Prezentarea PowerPoint pentru acest laborator poate fi găsită [aici](https://drive.google.com/file/d/1mjbkxFqYNB-lRRXGAhsdh1vK9nwbnDuS/view). În acest laborator veți avea niște exerciții ce folosesc PRFs, PRPs si DES. Puteți să vă uitați în cursul de [aici](https://drive.google.com/file/d/1Fjybv6k5QudRB1bkAi5shVUGlUyTO_0U/view) ca să vă aduceți aminte lucrurile de bază.

In [ ]:
import base64
from typing import Generator


def _pad(data: str, size: int) -> str:
    reminder = len(data) % size
    if reminder != 0:
        data = "0" * (size - reminder) + data
    return data


def _chunks(data: str, chunk_size: int) -> Generator[str, None, None]:
    data = _pad(data, chunk_size)
    for i in range(0, len(data), chunk_size):
        yield data[i : i + chunk_size]


def _hex(data: int) -> str:
    return format(data, "02x")


# Conversion functions


def byte_2_bin(bval: int) -> str:
    """Converts a byte value to a binary string.

    Args:
        bval (int):
            The byte value to be converted. It should be an integer between
            0 and 255.

    Returns:
        str: The binary string representation of the byte value, where each bit
        is encoded as a character. The result has a fixed length of 8 characters
        and is padded with leading zeros if necessary.

    Examples:
        >>> byte_2_bin(72)
        '01001000'
        >>> byte_2_bin(66)
        '01000010'
    """
    return bin(bval)[2:].zfill(8)


def hex_2_bin(data: str) -> str:
    """Converts a hexadecimal string to a binary representation.

    Args:
        data (str): The hexadecimal string to be converted. It should have an
            even number of characters and only contain valid hexadecimal digits
            (0-9, A-F, a-f).

    Returns:
        str: The binary representation of the hexadecimal string, where each
            pair of hexadecimal digits is encoded as an 8-bit binary number.

    Examples:
        >>> hex_2_bin("01abcd")
        '000000011010101111001101'
        >>> hex_2_bin("0a")
        '00001010'
    """
    return "".join(f"{int(x, 16):08b}" for x in _chunks(data, 2))


def bin_2_hex(data: str) -> str:
    """Converts a binary string to a hexadecimal representation.

    Args:
        data (str): The binary string to be converted. It should have a multiple
            of 8 characters and only contain valid binary digits (0 or 1).

    Returns:
        str: The hexadecimal representation of the binary string, where each
            group of 8 binary digits is encoded as a pair of hexadecimal digits.

    Examples:
        >>> bin_2_hex("000000011010101111001101")
        '01abcd'
        >>> bin_2_hex("00001010")
        '0a'
    """
    return "".join(f"{int(b, 2):02x}" for b in _chunks(data, 8))


def str_2_bin(data: str) -> str:
    """Converts a string to a binary representation.

    Args:
        data (str): The string to be converted.

    Returns:
        str: The binary representation of the string, where each character is
            encoded as an 8-bit binary number.

    Examples:
        >>> str_2_bin("Hello")
        '0100100001100101011011000110110001101111'
        >>> str_2_bin("IC")
        '0100100101000011'
    """
    return "".join(f"{ord(c):08b}" for c in data)


def bin_2_str(data: str) -> str:
    """Converts a binary string to a string.

    Args:
        data (str): The binary string to be converted. It should have a multiple
            of 8 characters and only contain valid binary digits (0 or 1).

    Returns:
        str: The string representation of the binary string, where each group
            of 8 binary digits is decoded as a character.

    Examples:
        >>> bin_2_str("0100100001100101011011000110110001101111")
        'Hello'
        >>> bin_2_str("0100100101000011")
        'IC'
    """
    return "".join(chr(int(b, 2)) for b in _chunks(data, 8))


def str_2_hex(data: str) -> str:
    """Converts a string to a hexadecimal representation.

    Args:
        data (str): The string to be converted.

    Returns:
        str: The hexadecimal representation of the string, where each character
            is encoded as a pair of hexadecimal digits.

    Examples:
        >>> str_2_hex("Hello")
        '48656c6c6f'
        >>> str_2_hex("IC")
        '4943'
    """
    return "".join(f"{ord(c):02x}" for c in data)


def hex_2_str(data: str) -> str:
    """Converts a hexadecimal string to a string.

    Args:
        data (str): The hexadecimal string to be converted. It should have an
            even number of characters and only contain valid hexadecimal digits
            (0-9, A-F, a-f).

    Returns:
        str: The string representation of the hexadecimal string, where each
            pair of hexadecimal digits is decoded as a character.

    Examples:
        >>> hex_2_str("48656c6c6f")
        'Hello'
        >>> hex_2_str("4943")
        'IC'
    """
    return "".join(chr(int(x, 16)) for x in _chunks(data, 2))


# XOR functions


def strxor(operand_1: str, operand_2: str) -> str:
    """Performs a bitwise exclusive OR (XOR) operation on two strings.

    Args:
        operand_1 (str): The first string to be XORed.
        operand_2 (str): The second string to be XORed.

    Returns:
        str: The result of the XOR operation on the two strings, where each
            character is encoded as an 8-bit binary number. The result has
            the same length as the shorter input string.

    Examples:
        >>> strxor("Hello", "IC")
        '\\x01&'
        >>> strxor("secret", "key")
        '\\x18\\x00\\x1a'
    """
    return "".join(chr(ord(x) ^ ord(y)) for (x, y) in zip(operand_1, operand_2))


def bitxor(operand_1: str, operand_2: str) -> str:
    """Performs a bitwise exclusive OR (XOR) operation on two bit-strings.

    Args:
        operand_1 (str): The first bit-string to be XORed. It should only
            contain valid binary digits (0 or 1).
        operand_2 (str): The second bit-string to be XORed. It should only
            contain valid binary digits (0 or 1).

    Returns:
        str: The result of the XOR operation on the two bit-strings, where each
            bit is encoded as a character. The result has the same length as
            the shorter input bit-string.

    Examples:
        >>> bitxor("01001000", "01000010")
        '00001010'
        >>> bitxor("10101010", "00110011")
        '10011001'
    """
    return "".join(str(int(x) ^ int(y)) for (x, y) in zip(operand_1, operand_2))


def hexxor(operand_1: str, operand_2: str) -> str:
    """Performs a bitwise exclusive OR (XOR) operation on two hexadecimal
    strings.

    Args:
        operand_1 (str): The first hexadecimal string to be XORed. It should
            have an even number of characters and only contain valid hexadecimal
            digits (0-9, A-F, a-f).
        operand_2 (str): The second hexadecimal string to be XORed. It should
            have an even number of characters and only contain valid
            digits (0-9, A-F, a-f).

    Returns:
        str: The result of the XOR operation on the two hexadecimal strings,
            where each pair of hexadecimal digits is encoded as a pair of
            hexadecimal digits. The result has the same length as the shorter
            input hexadecimal string.

    Examples:
        >>> hexxor("48656c6c6f", "42696e67")
        '0a0c020b'
        >>> hexxor("736563726574", "6b6579")
        '18001a'
    """
    return "".join(
        _hex(int(x, 16) ^ int(y, 16))
        for (x, y) in zip(_chunks(operand_1, 2), _chunks(operand_2, 2))
    )


# Python3 'bytes' functions


def bytes_to_string(bytes_data: bytearray | bytes) -> str:
    """Converts a byte array or a byte string to a string.

    Args:
        bytes_data (bytearray | bytes): The byte array or the byte string to be
            converted. It should be encoded in Latin-1 format.

    Returns:
        str: The string representation of the byte array or the byte string,
            decoded using Latin-1 encoding.

    Examples:
        >>> bytes_to_string(b'Hello')
        'Hello'
        >>> bytes_to_string(bytearray(b'IC'))
        'IC'
    """
    return bytes_data.decode(encoding="raw_unicode_escape")


def string_to_bytes(string_data: str) -> bytes:
    """Converts a string to a byte string.

    Args:
        string_data (str): The string to be converted.

    Returns:
        bytes: The byte string representation of the string, encoded using
        Latin-1 encoding.

    Examples:
        >>> string_to_bytes('Hello')
        b'Hello'
        >>> string_to_bytes('IC')
        b'IC'
    """
    return string_data.encode(encoding="raw_unicode_escape")


# Base64 functions


def b64encode(data: str) -> str:
    """Encodes a string to base64.

    Parameters:
        data (str): The string to be encoded.

    Returns:
        str: The base64 encoded string, using Latin-1 encoding.

    Examples:
        >>> b64encode("Hello")
        'SGVsbG8='
        >>> b64encode("IC")
        'SUM='
    """
    return bytes_to_string(base64.b64encode(string_to_bytes(data)))


def b64decode(data: str) -> str:
    """Decodes a base64 encoded string.

    Args:
        data (str): The base64 encoded string to be decoded. It should only
            contain valid base64 characters (A-Z, a-z, 0-9, +, /, =).

    Returns:
        str: The decoded string, using Latin-1 encoding.

    Examples:
        >>> b64decode("SGVsbG8=")
        'Hello'
        >>> b64decode("SUM=")
        'IC'
    """
    return bytes_to_string(base64.b64decode(string_to_bytes(data)))

## Exercițiul 1 (4p)

Fie $F : K × X \to Y$ un $PRF$ sigur cu $K = X = Y = \{0, 1\}^{n}$.
  * a) Arătați că $F_1(k,x) = F(k,x) \| 0$ nu e un $PRF$ sigur.
  * b) Arătați că $F_2(k, x) = F \left(k, x \oplus 1^{n}\right)$ este un $PRF$ sigur.

> **Hint:** Ca să arătați că e sigur, folosiți metoda reducerii la absurd: existența unui adversar $A$ care sparge $F_2$ implică existența unui alt adversar $B$ care sparge $F$ într-un timp similar.

  * c) Fie $K_3 = \{0, 1\}^{n+1}$. Construiți un nou $PRF$ $F_3 : K_3 \times X \to Y$ care să aibă urmatoarea proprietate: $PRF$-ul $F_3$ este sigur, dar dacă adversarul află ultimul bit din cheie atunci $PRF$-ul nu mai e sigur. Puteți astfel observa că până și aflarea unui singur bit din cheia secretă poate compromite siguranța unui $PRF$ .


> **Hint:** Fie $k_3 = k \| b$ unde $k \in \{0,1\}^{n}$ și $b \in \{0,1\}$. Definiți $F_3(k_3,x)$ ca fiind identic cu $F (k, x)$ pentru toate $x \neq 0^{n}$. Acum trebuie să mai definim $F_3\left(k_3, 0^{n}\right)$ astfel încat $F_3$ este un PRF sigur, dar poate fi diferențiat de o funcție aleatoare dacă ultimul bit din cheie este cunoscut. Arătați că $F_3$ e sigur prin reducere la absurd.

  * d) Construiți un nou $PRF$ $F_4 : K × X \to Y$ care rămâne sigur chiar și atunci când atacatorul cunoaște unul (oricare) dintre biții cheii. Funcția voastră $F_4$ trebuie să cheme o singură dată funcția $F$. Explicați de ce râmane un $PRF$ sigur chiar și după această scurgere de informație.

> Pentru două șiruri de biți $y$ și $z$ folosim $y \| z$ pentru a indica concatenarea între $y$ și $z$.


a) Adversarul va alege astfel:
F1 daca ultimul bit este 0
o functie complet aleatoare altfel

astfel reuseste sa obtina P(A cu F1) = 1(fiind mereu 0 va castiga mereu) altfel P(A aleator) = 1/2. astfel avantajul este |1-1/2| = 1/2 deci PRF-ul nu este sigur.

b) deja facut la lab

c) Conform cu hintul, o functie F3 complet identica cu F inafara de x = 0^n este un punct de plecare bun. Cea mai simpla solutie este sa definesc F3(k, 0^n) astfel: F(k, 0^n) daca b = 0 si F(k, 0^n) ⊕ 1^n daca b = 1. Astfel cu b necunoscut adversarul nu va realiza ca exista o diferenta, XOR pastrand uniformitatea aleatoare a lui F. Dar cand A stie la ce se asteapta(cunoaste b), atunci F3 nu mai poate fi considerata cu adevarat aleatoare. deci PRF-ul nu mai este sigur.

d)F4(k,x) = F(k,x) ⊕ H(k)
unde H este suma cifrelor cheii k in baza 10 hash-uite.
Cu toate ca se va cunoaste 1 bit din k, H(k) depinde de toti bitii, si arata complet aleator pentru Adversar. Deoarece suma este cat de cat corelata cu lungimea k am ales sa aplic si un hash(simplu sau complex) pentru a elimina aceasta nisa de atac.

## Exercițiile 2-3-4

Vom analiza acum rețele substituție-permutare (SPN - substitution-permutation networks).

### SPN 1 (3p)


Avem următorul SPN din figură:

![SPN](https://ocw.cs.pub.ro/courses/_media/sasc/laboratoare/spn_1r_reduced_2s.png)

unde S este un S-box din AES (detalii vor fi prezentate la curs), iar 'Permutation' este un bloc care doar șiftează toți biții cu 4 poziții la dreapta în mod ciclic. Atât S-boxurile cât și permutarea sunt inversabile și cunoscute de atacator (de voi). Fiecare intrare (x1, x2) are 8 biți (1 byte). Cheile k1, k2, și ieșirile y1, y2 au de asemenea dimensiunea de 1 byte.

1. Cum puteți determina cheia?
2. Fiind dată o pereche mesaj/ciphertext ('Hi' - char, 0xba52 - hex), găsiți k1 și k2. Afișați-le în ASCII.

> Puteți porni de la urmatorul schelet de cod:




In [ ]:
# fmt: off
# Rijndael S-box
sbox = [0x63, 0x7c, 0x77, 0x7b, 0xf2, 0x6b, 0x6f, 0xc5, 0x30, 0x01, 0x67,
        0x2b, 0xfe, 0xd7, 0xab, 0x76, 0xca, 0x82, 0xc9, 0x7d, 0xfa, 0x59,
        0x47, 0xf0, 0xad, 0xd4, 0xa2, 0xaf, 0x9c, 0xa4, 0x72, 0xc0, 0xb7,
        0xfd, 0x93, 0x26, 0x36, 0x3f, 0xf7, 0xcc, 0x34, 0xa5, 0xe5, 0xf1,
        0x71, 0xd8, 0x31, 0x15, 0x04, 0xc7, 0x23, 0xc3, 0x18, 0x96, 0x05,
        0x9a, 0x07, 0x12, 0x80, 0xe2, 0xeb, 0x27, 0xb2, 0x75, 0x09, 0x83,
        0x2c, 0x1a, 0x1b, 0x6e, 0x5a, 0xa0, 0x52, 0x3b, 0xd6, 0xb3, 0x29,
        0xe3, 0x2f, 0x84, 0x53, 0xd1, 0x00, 0xed, 0x20, 0xfc, 0xb1, 0x5b,
        0x6a, 0xcb, 0xbe, 0x39, 0x4a, 0x4c, 0x58, 0xcf, 0xd0, 0xef, 0xaa,
        0xfb, 0x43, 0x4d, 0x33, 0x85, 0x45, 0xf9, 0x02, 0x7f, 0x50, 0x3c,
        0x9f, 0xa8, 0x51, 0xa3, 0x40, 0x8f, 0x92, 0x9d, 0x38, 0xf5, 0xbc,
        0xb6, 0xda, 0x21, 0x10, 0xff, 0xf3, 0xd2, 0xcd, 0x0c, 0x13, 0xec,
        0x5f, 0x97, 0x44, 0x17, 0xc4, 0xa7, 0x7e, 0x3d, 0x64, 0x5d, 0x19,
        0x73, 0x60, 0x81, 0x4f, 0xdc, 0x22, 0x2a, 0x90, 0x88, 0x46, 0xee,
        0xb8, 0x14, 0xde, 0x5e, 0x0b, 0xdb, 0xe0, 0x32, 0x3a, 0x0a, 0x49,
        0x06, 0x24, 0x5c, 0xc2, 0xd3, 0xac, 0x62, 0x91, 0x95, 0xe4, 0x79,
        0xe7, 0xc8, 0x37, 0x6d, 0x8d, 0xd5, 0x4e, 0xa9, 0x6c, 0x56, 0xf4,
        0xea, 0x65, 0x7a, 0xae, 0x08, 0xba, 0x78, 0x25, 0x2e, 0x1c, 0xa6,
        0xb4, 0xc6, 0xe8, 0xdd, 0x74, 0x1f, 0x4b, 0xbd, 0x8b, 0x8a, 0x70,
        0x3e, 0xb5, 0x66, 0x48, 0x03, 0xf6, 0x0e, 0x61, 0x35, 0x57, 0xb9,
        0x86, 0xc1, 0x1d, 0x9e, 0xe1, 0xf8, 0x98, 0x11, 0x69, 0xd9, 0x8e,
        0x94, 0x9b, 0x1e, 0x87, 0xe9, 0xce, 0x55, 0x28, 0xdf, 0x8c, 0xa1,
        0x89, 0x0d, 0xbf, 0xe6, 0x42, 0x68, 0x41, 0x99, 0x2d, 0x0f, 0xb0,
        0x54, 0xbb, 0x16]

# Rijndael Inverted S-box
rsbox = [0x52, 0x09, 0x6a, 0xd5, 0x30, 0x36, 0xa5, 0x38, 0xbf, 0x40, 0xa3,
         0x9e, 0x81, 0xf3, 0xd7, 0xfb, 0x7c, 0xe3, 0x39, 0x82, 0x9b, 0x2f,
         0xff, 0x87, 0x34, 0x8e, 0x43, 0x44, 0xc4, 0xde, 0xe9, 0xcb, 0x54,
         0x7b, 0x94, 0x32, 0xa6, 0xc2, 0x23, 0x3d, 0xee, 0x4c, 0x95, 0x0b,
         0x42, 0xfa, 0xc3, 0x4e, 0x08, 0x2e, 0xa1, 0x66, 0x28, 0xd9, 0x24,
         0xb2, 0x76, 0x5b, 0xa2, 0x49, 0x6d, 0x8b, 0xd1, 0x25, 0x72, 0xf8,
         0xf6, 0x64, 0x86, 0x68, 0x98, 0x16, 0xd4, 0xa4, 0x5c, 0xcc, 0x5d,
         0x65, 0xb6, 0x92, 0x6c, 0x70, 0x48, 0x50, 0xfd, 0xed, 0xb9, 0xda,
         0x5e, 0x15, 0x46, 0x57, 0xa7, 0x8d, 0x9d, 0x84, 0x90, 0xd8, 0xab,
         0x00, 0x8c, 0xbc, 0xd3, 0x0a, 0xf7, 0xe4, 0x58, 0x05, 0xb8, 0xb3,
         0x45, 0x06, 0xd0, 0x2c, 0x1e, 0x8f, 0xca, 0x3f, 0x0f, 0x02, 0xc1,
         0xaf, 0xbd, 0x03, 0x01, 0x13, 0x8a, 0x6b, 0x3a, 0x91, 0x11, 0x41,
         0x4f, 0x67, 0xdc, 0xea, 0x97, 0xf2, 0xcf, 0xce, 0xf0, 0xb4, 0xe6,
         0x73, 0x96, 0xac, 0x74, 0x22, 0xe7, 0xad, 0x35, 0x85, 0xe2, 0xf9,
         0x37, 0xe8, 0x1c, 0x75, 0xdf, 0x6e, 0x47, 0xf1, 0x1a, 0x71, 0x1d,
         0x29, 0xc5, 0x89, 0x6f, 0xb7, 0x62, 0x0e, 0xaa, 0x18, 0xbe, 0x1b,
         0xfc, 0x56, 0x3e, 0x4b, 0xc6, 0xd2, 0x79, 0x20, 0x9a, 0xdb, 0xc0,
         0xfe, 0x78, 0xcd, 0x5a, 0xf4, 0x1f, 0xdd, 0xa8, 0x33, 0x88, 0x07,
         0xc7, 0x31, 0xb1, 0x12, 0x10, 0x59, 0x27, 0x80, 0xec, 0x5f, 0x60,
         0x51, 0x7f, 0xa9, 0x19, 0xb5, 0x4a, 0x0d, 0x2d, 0xe5, 0x7a, 0x9f,
         0x93, 0xc9, 0x9c, 0xef, 0xa0, 0xe0, 0x3b, 0x4d, 0xae, 0x2a, 0xf5,
         0xb0, 0xc8, 0xeb, 0xbb, 0x3c, 0x83, 0x53, 0x99, 0x61, 0x17, 0x2b,
         0x04, 0x7e, 0xba, 0x77, 0xd6, 0x26, 0xe1, 0x69, 0x14, 0x63, 0x55,
         0x21, 0x0c, 0x7d]
# fmt: on


def permute4(s: str) -> str:
    """Perform a permutatation by shifting all bits 4 positions right.

    Args:
        s (str): The input is assumed to be a 16-bit bitstring.

    Raises:
        TypeError: Expected input to be a string.
        ValueError: Expected input to have length 16.

    Returns:
        str: The permuted string by shifting all bits 4 positions right.
    """
    if not isinstance(s, str):
        raise TypeError(
            f"Invalid type for the input. Expected `str`, but got `{type(s)}`."
        )
    if len(s) != 16:
        raise ValueError(
            f"Invalit input length. Expected to be 16, but got {len(s)}."
        )
    return s[12:16] + s[0:12]


def permute_inv4(s: str) -> str:
    """Perform the inverse of permute4.

    Args:
        s (str): The input is assumed to be a 16-bit bitstring.

    Raises:
        TypeError: Expected input to be a string.
        ValueError: Expected input to have length 16.

    Returns:
        str: The permuted string by shifting all bits 4 positions left.
    """
    if not isinstance(s, str):
        raise TypeError(
            f"Invalid type for the input. Expected `str`, but got `{type(s)}`."
        )
    if len(s) != 16:
        raise ValueError(
            f"Invalit input length. Expected to be 16, but got {len(s)}."
        )
    return s[4:16] + s[0:4]


def spn_1r_reduced_2s(k: str, x: str) -> str:
    """Performs an encryption with a substitution-permutation network.

    Args:
        k (str): Key k = {k1, k2}, total of 16 bits (2 x 8 bits). Assumed to be
            bitstring.
        x (str): Input x = {x1, x2}, total of 16 bits (2 x 8 bits). Assumed to
            be bitstring.

    Returns:
        str: a 16-bit bitstring containing the encryption y = {y1, y2}
    """
    # Split input and key
    x1, x2 = x[0:8], x[8:16]
    k1, k2 = k[0:8], k[8:16]

    # Apply S-box
    u1 = bitxor(x1, k1)
    v1 = sbox[int(u1, 2)]
    v1 = byte_2_bin(v1)

    u2 = bitxor(x2, k2)
    v2 = sbox[int(u2, 2)]
    v2 = byte_2_bin(v2)

    # Apply permutation
    pin = v1 + v2
    pout = permute4(pin)

    return pout


def spn_1r_full_2s(k: str, x: str) -> str:
    """Performs an encryption with a substitution-permutation network.

    Args:
        k (str): Key k = {k1, k2, k3, k4}, total of 32 bits (4 x 8 bits).
            Assumed to be bitstring.
        x (str): Input x = {x1, x2}, total of 16 bits (2 x 8 bits).
            Assumed to be bitstring.

    Returns:
        str: a 16-bit bitstring containing the encryption y = {y1, y2}
    """
    # Split input and key
    x1, x2 = x[0:8], x[8:16]
    k1, k2, k3, k4 = k[0:8], k[8:16], k[16:24], k[24:32]

    # Apply S-box
    u1 = bitxor(x1, k1)
    v1 = sbox[int(u1, 2)]
    v1 = byte_2_bin(v1)

    u2 = bitxor(x2, k2)
    v2 = sbox[int(u2, 2)]
    v2 = byte_2_bin(v2)

    # Apply permutation
    pin = v1 + v2
    pout = permute4(pin)

    # Apply final XOR
    po1 = pout[0:8]
    po2 = pout[8:16]
    y1 = bitxor(po1, k3)
    y2 = bitxor(po2, k4)

    return y1 + y2

In [ ]:
# Run reduced 2-byte SPN
msg = "Hi"
key = "ab"  # Find the key that should replace ??
ks = str_2_bin(key)

# TODO 1: Find the key in the reduced 2-byte SPN
def recover_key(ciphertext: str, plaintext: str) -> str:
    """Recover the key from a given plaintext-ciphertext pair."""
    # Convert inputs to binary
    xs = str_2_bin(plaintext)
    ys = format(int(ciphertext, 16), '016b')

    # Inverse permutation
    pin = permute_inv4(ys)
    v1, v2 = pin[:8], pin[8:]

    # Inverse S-box
    u1 = byte_2_bin(rsbox[int(v1, 2)])
    u2 = byte_2_bin(rsbox[int(v2, 2)])

    # Recover key
    x1, x2 = xs[:8], xs[8:]
    k1 = bitxor(x1, u1)
    k2 = bitxor(x2, u2)

    return k1 + k2

# Given values
ciphertext = "ba52"
plaintext = "Hi"

# Recover the key
recovered_key = recover_key(ciphertext, plaintext)
print("Recovered key (binary):", recovered_key)
print("Recovered key (hex):", bin_2_hex(recovered_key[:8]), bin_2_hex(recovered_key[8:]))
key_str = bytes.fromhex(bin_2_hex(recovered_key)).decode("ascii")
print("Recovered key (text):", key_str)


xs = str_2_bin(msg)
ys = spn_1r_reduced_2s(ks, xs)
print(
    "Two y halves of reduced SPN:"
    f" {ys[0:8]} (hex: {bin_2_hex(ys[0:8])}),"
    f" {ys[8:16]} (hex: {bin_2_hex(ys[8:16])})"
)

### SPN 2 (3p)

Vom folosi acum un SPN mai mai bun unde rezultatul permutarilor este XOR-at cu alți 2 octeți din cheie k3 și k4, ca în figură:

![spn2](https://ocw.cs.pub.ro/courses/_media/sasc/laboratoare/spn_1r_full_2s.png)

 1. Găsiți cheia știind următoarele perechi mesaj/ciphertext: ('Om', 0x0073), ('El', 0xd00e), ('an', 0x855b). Afișați-o în ASCII

> O să trebuiască să faceți căutare brute-force.

In [ ]:
# Run full 2-byte SPN
msg = "Om"
key = "abcd"  # Find the key that should replace ????
ks = str_2_bin(key)

# TODO 2: Find the key in the full 2-byte SPN
import itertools

def find_keys_partial(ciphertext, plaintext):
    """Finds the first two key bytes by partially decrypting the ciphertext."""
    #prima jumatate de cheie care poate fi aflata calculand fara bruteforce
    s = permute_inv4(hex_2_bin(ciphertext))
    xored_plaintext1 = rsbox[int(s[:8], 2)]
    xored_plaintext2 = rsbox[int(s[8:16], 2)]

    k1 = bitxor(byte_2_bin(xored_plaintext1), str_2_bin(plaintext[0]))
    k2 = bitxor(byte_2_bin(xored_plaintext2), str_2_bin(plaintext[1]))

    return k1, k2

def find_keys_full(ciphertexts, plaintexts):
    """Finds all four key bytes by iterating through possible values for k3 and k4."""
    #cealalta jumatate in schimb trebuie facuta incercand toate optiunile
    for k3, k4 in itertools.product(range(256), repeat=2):
        xored_ciphertexts = [hexxor(f"{k3:02x}{k4:02x}", ct) for ct in ciphertexts]

        k1, k2 = find_keys_partial(xored_ciphertexts[0], plaintexts[0])
        full_key = k1 + k2 + byte_2_bin(k3) + byte_2_bin(k4)

        if all(spn_1r_full_2s(full_key, str_2_bin(pt)) == hex_2_bin(ct)
               for pt, ct in zip(plaintexts[1:], ciphertexts[1:])):
            return full_key

    raise ValueError("No valid key found.")

msgs = ["Om", "El", "an"]
ciphertexts = ["0073", "d00e", "855b"]

key_bits = find_keys_full(ciphertexts, msgs)
key_str = bytes.fromhex(bin_2_hex(key_bits)).decode("ascii")

print(f"The key bits: {key_bits}")
print(f'The str key is "{key_str}"')
xs = str_2_bin(msg)
ys = spn_1r_full_2s(ks, xs)
print(
    "Two y halves of full SPN (2 bytes):"
    f" {ys[0:8]} (hex: {bin_2_hex(ys[0:8])}),"
    f" {ys[8:16]} (hex: {bin_2_hex(ys[8:16])})"
)

### SPN 3 - Bonus (2p)


În acest exercițiu vom folosi un bloc mai mare. În acest SPN sunt introduși 4 octeți x=[x1 || x2 || x3 || x4], iar cheia folosită conține 8 octeți k=[k1 || k2 || k3 || k4 || k5 || k6 || k7 || k8] ca în figură:

![SPN3](https://ocw.cs.pub.ro/courses/_media/sasc/laboratoare/spn_1r_full_4s.png)

Ca și în SPN-ul precedent, permutarea rotește toți octeții la dreapta cu 4 biți.

1. Găsiți cheia știind următoarele perechi mesaj/ciphertext: ('Omul', 0xddcf7bc7), ('stea', 0x96d58b43), ('luna', 0x9c3f2303)

> De data aceasta nu se poate executa o cautare brute-force pe toți octeții de XOR. Încercați să atacați câte un S-box pe rând. Gandiți-vă ce biți sunt afectați de fiecare S-box pentru a construi un atac eficient. Ce se întâmplă dacă faceți brute force pe k5 și k6? Puteți să aflați k1?


In [ ]:
# TODO 3: Implement the SPN, starting from the above code and find the key


